In [2]:
import os

import numpy as np
import pandas as pd
import scipy
import seaborn as sns
from matplotlib import pyplot as plt

sns.set()

In [3]:
# project files
from rp_model.files import set_files_directory

set_files_directory("./files")

from rp_model.calc import (
    game, FitOptions, download_data, refresh_pokedex, make_precomputed_columns, make_initial_guess
)
from rp_model.utils import (
    pack, truncate, lookup_table
)

In [4]:
# stuff for display

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 120)

In [5]:
refresh_rp_file = False

if refresh_rp_file:
    refresh_pokedex()
    data = download_data()
    data.to_pickle(FitOptions.data_file)
else:
    data = pd.read_pickle(FitOptions.data_file)

In [6]:
data.describe()
data.head()

,Level,RP,MS lvl,ModelRP,Difference,Freq1,Berry1,Ing1P,MiscMult,NrgNat,Helps per hour,IngrMult,SkillMult,BerryD,IngD,SklVal,FreqL,HelpNat,IngrNat,Ingr%,SkillNat,Skill%,BerryL,Dupes,Diff -0.1% ingr,Diff +0.1% ingr,Diff,Amnt,Ing2P,RP Multiplier,HelpSub,IngrSub,SkillSub
count,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.0,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000,11552.000000
mean,15.044754,805.785838,1.681267,805.786357,0.000519,3764.447715,28.172611,108.696849,1.038042,0.993608,1.102492,0.211107,0.038624,65.051073,155.067954,986.666292,3571.434882,0.987457,0.999013,0.203712,1.009159,0.036323,42.905125,1.0,-0.642046,0.632292,-0.004764,0.261427,12.721780,1.045447,0.981318,1.031787,1.041416
std,10.121125,629.277573,1.158559,629.277478,0.022785,1012.726082,2.734838,15.892038,0.095506,0.044132,0.347689,0.058416,0.021864,38.854113,70.321507,916.640264,1080.443110,0.056911,0.117379,0.040569,0.120121,0.018792,12.596589,0.0,0.787411,0.791723,0.288472,0.882230,40.051497,0.086528,0.045257,0.095826,0.108286
min,1.000000,245.000000,1.000000,245.000000,0.000000,2100.000000,24.000000,90.000000,0.920000,0.920000,0.520000,0.100800,0.006400,25.000000,90.000000,400.000000,1407.780000,0.900000,0.800000,0.126000,0.800000,0.008000,24.000000,1.0,-8.430000,-1.180000,-0.500000,0.000000,0.000000,1.000000,0.790000,1.000000,1.000000
25%,8.000000,394.000000,1.000000,394.000000,0.000000,2800.000000,26.000000,98.000000,1.000000,1.000000,0.820000,0.169600,0.021000,39.000000,101.000000,400.000000,2656.800000,1.000000,1.000000,0.174000,1.000000,0.021000,34.000000,1.0,-0.910000,0.140000,-0.250000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
50%,12.000000,583.000000,1.000000,583.500000,0.000000,3700.000000,28.000000,103.000000,1.000000,1.000000,1.040000,0.203000,0.036000,53.000000,121.000000,880.000000,3430.000000,1.000000,1.000000,0.201000,1.000000,0.036000,40.000000,1.0,-0.500000,0.480000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
75%,21.000000,946.000000,2.000000,946.000000,0.000000,4400.000000,31.000000,121.000000,1.080000,1.000000,1.350000,0.247900,0.051600,76.000000,206.000000,1083.000000,4356.000000,1.000000,1.000000,0.230000,1.000000,0.047000,48.000000,1.0,-0.160000,0.910000,0.240000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,55.000000,5473.000000,7.000000,5473.000000,1.000000,6300.000000,35.000000,151.000000,1.550000,1.080000,2.550000,0.517400,0.217100,354.000000,511.000000,12438.000000,6916.140000,1.100000,1.200000,0.339000,1.200000,0.141000,118.000000,1.0,0.950000,7.840000,0.510000,8.000000,342.000000,1.442000,1.000000,1.540000,1.540000


,Pokemon,Level,RP,NatureEN/ZH,MS lvl,Source,ModelRP,Difference,Nature,ID,NatureP,NatureN,Type,Class,MSkill,Freq1,Berry1,Ing1,Ing1P,MiscMult,NrgNat,Helps per hour,IngrMult,SkillMult,BerryD,IngD,SklVal,FreqL,HelpNat,IngrNat,Ingr%,SkillNat,Skill%,BerryL,Dupes,DupeMatch,Diff -0.1% ingr,Diff +0.1% ingr,Diff,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,RP Multiplier,HelpSub,IngrSub,SkillSub,Sub Skill 3
0,Arbok,8.0,531.0,Hardy,1.0,Questions help guides,531.0,0.0,Neutral,ArbokNeutral81,-,-,Poison,Berries,Charge Energy S,3700.0,32.0,Bean Sausage,103.0,1.00,1.00,0.98,0.2640,0.0570,78.0,103.0,400.0,3648.20,1.0,1.0,0.26,1.0,0.057,39,1,True,-0.4900,-0.2200,-0.3600,,,,0.0,0.0,1.0,1.0,1.0,1.0,
1,Arbok,8.0,538.0,Quiet,1.0,Questions help guides,538.0,0.0,Quiet,ArbokQuiet81,Ingredient Finding,Exp Gains,Poison,Berries,Charge Energy S,3700.0,32.0,Bean Sausage,103.0,1.00,1.00,0.98,0.3168,0.0570,78.0,103.0,400.0,3648.20,1.0,1.2,0.26,1.0,0.057,39,1,True,-0.1400,0.1300,-0.0100,,,,0.0,0.0,1.0,1.0,1.0,1.0,
2,Arbok,9.0,528.0,Bold,1.0,,528.0,0.0,Bold,ArbokBold91,Energy Recovery,Speed of Help,Poison,Berries,Charge Energy S,3700.0,32.0,Bean Sausage,103.0,1.08,1.08,0.89,0.2640,0.0570,80.0,103.0,400.0,4004.88,1.1,1.0,0.26,1.0,0.057,40,1,True,0.1956,0.4548,0.3252,,,,0.0,0.0,1.0,1.0,1.0,1.0,
3,Arbok,9.0,539.0,Hardy,1.0,Double Jumper,539.0,0.0,Neutral,ArbokNeutral91,-,-,Poison,Berries,Charge Energy S,3700.0,32.0,Bean Sausage,103.0,1.00,1.00,0.98,0.2640,0.0570,80.0,103.0,400.0,3640.80,1.0,1.0,0.26,1.0,0.057,40,1,True,-0.4700,-0.2000,-0.3400,,,,0.0,0.0,1.0,1.0,1.0,1.0,
4,Arbok,9.0,574.0,Naughty,1.0,Rate My Mon,574.0,0.0,Naughty,ArbokNaughty91,Speed of Help,Main Skill Chance,Poison,Berries,Charge Energy S,3700.0,32.0,Bean Sausage,103.0,1.00,1.00,1.09,0.2640,0.0456,80.0,103.0,400.0,3276.72,0.9,1.0,0.26,0.8,0.057,40,1,True,0.1200,0.4100,0.2600,,,,0.0,0.0,1.0,1.0,1.0,1.0,


In [7]:
# Use existing columns to reproduce RP components with manually fitted %
data["__IngGrowth__"] = \
    1.0 + 0.01 * lookup_table(data["Level"],game.data.ing_growth,"Level","Ing Growth")
data["__SheetIng__"]  = \
    truncate(5 * data["Helps per hour"] * data["IngrMult"] * data["IngD"] * data["__IngGrowth__"] , 2)
data["__SheetBerry__"] = \
    truncate( 5 * data["Helps per hour"] * (1.0 - data["IngrMult"]) * data["BerryD"], 2)
data["__SheetSkill__"] = \
    truncate( 5 * data["Helps per hour"] * data["SkillMult"] * data["SklVal"], 2)

data["__FractionalRP __"] = \
    data["MiscMult"] * (data["__SheetIng__"] + data["__SheetBerry__"] + data["__SheetSkill__"])

data["__RoundRP __"] = data["__FractionalRP __"].round()

data["__CMP __"]  = data["__RoundRP __"]  - data["RP"]

In [8]:
data


,Pokemon,Level,RP,NatureEN/ZH,MS lvl,Source,ModelRP,Difference,Nature,ID,NatureP,NatureN,Type,Class,MSkill,Freq1,Berry1,Ing1,Ing1P,MiscMult,NrgNat,Helps per hour,IngrMult,SkillMult,BerryD,IngD,SklVal,FreqL,HelpNat,IngrNat,Ingr%,SkillNat,Skill%,BerryL,Dupes,DupeMatch,Diff -0.1% ingr,Diff +0.1% ingr,Diff,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,RP Multiplier,HelpSub,IngrSub,SkillSub,Sub Skill 3,__IngGrowth__,__SheetIng__,__SheetBerry__,__SheetSkill__,__FractionalRP __,__RoundRP __,__CMP __
0,Arbok,8.0,531.0,Hardy,1.0,Questions help guides,531.0,0.0,Neutral,ArbokNeutral81,-,-,Poison,Berries,Charge Energy S,3700.0,32.0,Bean Sausage,103.0,1.00,1.00,0.98,0.2640,0.0570,78.0,103.0,400.0,3648.20,1.0,1.0,0.260,1.0,0.057,39,1,True,-0.4900,-0.2200,-0.3600,,,,0.0,0.0,1.000,1.00,1.00,1.00,,1.033,137.63,281.29,111.72,530.6400,531.0,0.0
1,Arbok,8.0,538.0,Quiet,1.0,Questions help guides,538.0,0.0,Quiet,ArbokQuiet81,Ingredient Finding,Exp Gains,Poison,Berries,Charge Energy S,3700.0,32.0,Bean Sausage,103.0,1.00,1.00,0.98,0.3168,0.0570,78.0,103.0,400.0,3648.20,1.0,1.2,0.260,1.0,0.057,39,1,True,-0.1400,0.1300,-0.0100,,,,0.0,0.0,1.000,1.00,1.00,1.00,,1.033,165.16,261.11,111.72,537.9900,538.0,0.0
2,Arbok,9.0,528.0,Bold,1.0,,528.0,0.0,Bold,ArbokBold91,Energy Recovery,Speed of Help,Poison,Berries,Charge Energy S,3700.0,32.0,Bean Sausage,103.0,1.08,1.08,0.89,0.2640,0.0570,80.0,103.0,400.0,4004.88,1.1,1.0,0.260,1.0,0.057,40,1,True,0.1956,0.4548,0.3252,,,,0.0,0.0,1.000,1.00,1.00,1.00,,1.039,125.72,262.01,101.46,528.3252,528.0,0.0
3,Arbok,9.0,539.0,Hardy,1.0,Double Jumper,539.0,0.0,Neutral,ArbokNeutral91,-,-,Poison,Berries,Charge Energy S,3700.0,32.0,Bean Sausage,103.0,1.00,1.00,0.98,0.2640,0.0570,80.0,103.0,400.0,3640.80,1.0,1.0,0.260,1.0,0.057,40,1,True,-0.4700,-0.2000,-0.3400,,,,0.0,0.0,1.000,1.00,1.00,1.00,,1.039,138.43,288.51,111.72,538.6600,539.0,0.0
4,Arbok,9.0,574.0,Naughty,1.0,Rate My Mon,574.0,0.0,Naughty,ArbokNaughty91,Speed of Help,Main Skill Chance,Poison,Berries,Charge Energy S,3700.0,32.0,Bean Sausage,103.0,1.00,1.00,1.09,0.2640,0.0456,80.0,103.0,400.0,3276.72,0.9,1.0,0.260,0.8,0.057,40,1,True,0.1200,0.4100,0.2600,,,,0.0,0.0,1.000,1.00,1.00,1.00,,1.039,153.97,320.89,99.40,574.2600,574.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,Wigglytuff,53.0,3670.0,溫順,6.0,Hakkk,3670.0,0.0,Gentle,WigglytuffGentle536Skill Trigger MInventory Up...,Main Skill Chance,Energy Recovery,Fairy,Skills,Energy for Everyone S,2900.0,26.0,Honey,101.0,1.04,0.92,1.38,0.2053,0.0652,94.0,171.0,5785.0,2598.40,1.0,1.0,0.174,1.2,0.040,94,1,True,-1.4400,1.3700,-0.0400,Skill Trigger M,Inventory Up M,Pure Oil,2.0,121.0,1.139,1.00,1.18,1.36,Ingredient Finder S,1.696,410.82,515.44,2602.55,3669.9624,3670.0,0.0
115,Wigglytuff,54.0,3687.0,溫順,6.0,Hakkk,3687.0,0.0,Gentle,WigglytuffGentle546Skill Trigger MInventory Up...,Main Skill Chance,Energy Recovery,Fairy,Skills,Energy for Everyone S,2900.0,26.0,Honey,101.0,1.04,0.92,1.38,0.2053,0.0652,96.0,171.0,5785.0,2592.60,1.0,1.0,0.174,1.2,0.040,96,1,True,-1.0000,1.8400,0.4100,Skill Trigger M,Inventory Up M,Pure Oil,2.0,121.0,1.139,1.00,1.18,1.36,Ingredient Finder S,1.720,416.64,526.40,2602.55,3687.4136,3687.0,0.0
116,Wigglytuff,55.0,3738.0,溫順,6.0,Hakkk,3738.0,0.0,Gentle,WigglytuffGentle556Skill Trigger MInventory Up...,Main Skill Chance,Energy Recovery,Fairy,Skills,Energy for Everyone S,2900.0,26.0,Honey,101.0,1.04,0.92,1.39,0.2053,0.0652,99.0,171.0,5785.0,2586.80,1.0,1.0,0.174,1.2,0.040,99,1,True,-1.7300,1.1500,-0.2800,Skill Trigger M,Inventory Up M,Pure Oil,2.0,121.0,1.139,1.00,1.18,1.36,Ingredient Finder S,1.745,425.76,546.79,2621.41,3737.7184,3738.0,0.0
117,Feraligatr,55.0,2920.0,頑皮,3.0,Hakkk,2920.0,0.0,Naughty,FeraligatrNaughty553Skill Trigger SIngredient ...,Speed of Help,Main Skill Chance,Water,Berries,Charge Strength S,2800.0,31.0,Bean S

In [9]:
x0, unpack_info = pack(*make_initial_guess())
computed = make_precomputed_columns(data)


In [10]:
from rp_model.calc import compute_rp_components

rp_comp = pd.DataFrame(
    compute_rp_components(x0, data, computed, unpack_info),
    index=data.index
)

rp_comp

,rp_model,ingredients_value_model,berries_value_model,main_skill_value_model,bonus_multipliers_model
0,531.0,137.63,281.29,111.72,1.00
1,538.0,165.16,261.11,111.72,1.00
2,528.0,125.72,262.01,101.46,1.08
3,539.0,138.43,288.51,111.72,1.00
4,574.0,153.97,320.89,99.40,1.00
...,...,...,...,...,...
114,3670.0,410.82,515.44,2602.55,1.04
115,3687.0,416.64,526.40,2602.55,1.04
116,3738.0,425.76,546.79,2621.41,1.04
117,2920.0,751.36,1842.21,325.93,1.00


In [12]:
data2 = pd.concat([data, rp_comp], axis=1)

data2["error"] = (data2["rp_model"] - data2["RP"]).abs()
data2 = data2[data2["error"]>0]
data2 = data2.sort_values("error", ascending=False)

data2


,Pokemon,Level,RP,NatureEN/ZH,MS lvl,Source,ModelRP,Difference,Nature,ID,NatureP,NatureN,Type,Class,MSkill,Freq1,Berry1,Ing1,Ing1P,MiscMult,NrgNat,Helps per hour,IngrMult,SkillMult,BerryD,IngD,SklVal,FreqL,HelpNat,IngrNat,Ingr%,SkillNat,Skill%,BerryL,Dupes,DupeMatch,Diff -0.1% ingr,Diff +0.1% ingr,Diff,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,RP Multiplier,HelpSub,IngrSub,SkillSub,Sub Skill 3,__IngGrowth__,__SheetIng__,__SheetBerry__,__SheetSkill__,__FractionalRP __,__RoundRP __,__CMP __,rp_model,ingredients_value_model,berries_value_model,main_skill_value_model,bonus_multipliers_model,error
324,Caterpie,4.0,279.0,Jolly,1.0,Rate My Mon,279.0,0.0,Jolly,CaterpieJolly41,EXP Gains,Ingredient Finding,Bug,Berries,Ingredient Magnet S,4400.0,24.0,Honey,101.0,1.00,1.00,0.82,0.1432,0.0080,54.0,101.0,880.0,4373.60,1.0,0.8,0.180,1.0,0.008,27,1,True,-0.7000,-0.3000,-0.5000,,,,0.0,0.0,1.00,1.00,1.00,1.00,,1.011,59.95,189.69,28.86,278.5000,278.0,-1.0,278.0,59.95,189.69,28.86,1.00,1.0
3493,Victreebel,9.0,670.0,Modest,2.0,ev,670.0,0.0,Modest,VictreebelModest92,Ingredient Finding,Speed of Help,Grass,Ingredients,Charge Energy S,2800.0,30.0,Snoozy Tomato,110.0,1.00,1.00,1.18,0.2796,0.0390,38.0,220.0,569.0,3030.72,1.1,1.2,0.230,1.0,0.039,38,1,True,-1.6300,0.6200,-0.5000,,,,0.0,0.0,1.00,1.00,1.00,1.00,,1.039,377.07,161.51,130.92,669.5000,669.0,-1.0,669.0,377.07,161.51,130.92,1.00,1.0
6414,Sylveon,24.0,825.0,Serious,1.0,https://www.youtube.com/watch?v=i-PBKUwF4Cg#t=...,825.0,0.0,Neutral,SylveonNeutral241Skill Trigger S,-,-,Fairy,Skills,Energy for Everyone S,2600.0,26.0,Moomoo Milk,98.0,1.00,1.00,1.45,0.1780,0.0472,49.0,98.0,1120.0,2480.40,1.0,1.0,0.178,1.0,0.040,49,1,True,-0.9800,-0.0100,-0.5000,Skill Trigger S,,,0.0,0.0,1.00,1.00,1.00,1.18,,1.180,149.23,292.01,383.26,824.5000,824.0,-1.0,824.0,149.23,292.01,383.26,1.00,1.0
6365,Swalot,17.0,739.0,爽朗,1.0,Hakkk,739.0,0.0,Jolly,SwalotJolly171Helping Speed M,EXP Gains,Ingredient Finding,Poison,Skills,Dream Shard Magnet S,3500.0,32.0,Greengrass Soybeans,100.0,1.00,1.00,1.23,0.1680,0.0700,48.0,100.0,880.0,2913.40,1.0,0.8,0.210,1.0,0.070,48,1,True,-0.8800,-0.1100,-0.5000,Helping Speed M,,,0.0,0.0,1.00,0.86,1.00,1.00,,1.104,114.06,245.60,378.84,738.5000,738.0,-1.0,738.0,114.06,245.60,378.84,1.00,1.0
5815,Slaking,27.0,1627.0,Sassy,3.0,,1627.0,0.0,Sassy,SlakingSassy273Helping Speed MSkill Trigger S,Main Skill Chance,EXP Gains,Normal,Berries,Ingredient Magnet S,3800.0,28.0,Snoozy Tomato,110.0,1.00,1.00,1.16,0.3390,0.0948,108.0,110.0,1726.0,3097.76,1.0,1.0,0.339,1.2,0.067,54,1,True,-0.6600,-0.3600,-0.5000,Helping Speed M,Skill Trigger S,,0.0,0.0,1.00,0.86,1.00,1.18,,1.218,263.43,414.05,949.02,1626.5000,1626.0,-1.0,1626.0,263.43,414.05,949.02,1.00,1.0
5688,Rattata,15.0,366.0,大膽,1.0,Hakkk,367.0,1.0,Bold,RattataBold151Ingredient Finder S,Energy Recovery,Speed of Help,Normal,Berries,Charge Energy S,4900.0,28.0,Fancy Apple,90.0,1.08,1.08,0.68,0.2796,0.0300,84.0,90.0,400.0,5239.08,1.1,1.0,0.237,1.0,0.030,42,1,True,0.4700,0.5700,0.5100,Ingredient Finder S,,,0.0,0.0,1.00,1.00,1.18,1.00,,1.085,92.82,205.74,40.80,366.5088,367.0,1.0,367.0,92.82,205.74,40.80,1.08,1.0
4176,Mankey,12.0,403.0,Bashful,1.0,https://discord.com/channels/11387018194643927...,403.0,0.0,Neutral,MankeyNeutral121Ingredient Finder S,-,-,Fighting,Berries,Charge Strength S,4200.0,27.0,Bean Sausage,103.0,1.00,1.00,0.87,0.2324,0.0220,76.0,103.0,400.0,4107.60,1.0,1.0,0.197,1.0,0.022,38,1,True,-0.6300,-0.3400,-0.4900,Ingredient Finder S,,,0.0,0.0,1.00,1.00,1.18,1.00,,1.061,110.47,253.76,38.27,402.5000,402.0,-1.0,402.0,110.47,253.76,38.27,1.00,1.0
3697,Igglybuff,15.0,431.0,Sassy,1.0,,431.0,0.0,Sassy,IgglybuffSassy151Skill Trigger M,Main Skill Chance,EXP Gains,Fairy,Skills,Energy for Everyone S,5200.0,26.0,Honey,101.0,1.00,1.00,0.71,0.1700,0.0620,40.0,101.0,1120.0,5054.40,1.0,1.0,0.170,1.2,0.038,40,1,True,-0.7500,-0.2600,-0.5000,Skill Trigger M,,,0.0,0.0,1.00,1.00,1.00,1.36,,1.085,66.13,117.86,246.51,430.5000,430.0,-1.0,430.0,66.13,117.86,246.51,1.00,1.0